Should change the format of this page to the following:

1. Gather User Input
2. Generate all required directories
3. Load in training data
4. Load in baseline models
5. Generate List of Pruned Models
6. Generate List of Quantized Models
7. Determine Accuracies of Baseline, Pruned, Quantized Models
8. Determine Size of Baseline, Pruned, Quantized Models
9. Save Accuracies and Sizes to a CSV
10. Plot Accuracy vs Size to a Figure and save it

### User Input

In [ ]:
pruning_user_params = {
    'model_depth': 7,        # 3, 5, 7
    'trial_number': 3,
    'prune layers': 'dense', # none, all, dense, or conv (does not work right now)
    'verbose_level': 0
}

### Path Setup

In [ ]:
depth = pruning_user_params['model_depth']
verbosity = pruning_user_params['verbose_level']
trial_n = pruning_user_params['trial_number']

mfccs_json_path         = "../mfccs_cnn_humpbackwhale_walrus_bowheadwhale_fin_finbackwhale_killerwhale_emptyocean.json"
saved_model_path        = f'../saved_model/layers{depth}/trial{trial_n}/'
pruned_model_path       = f'../saved_model/pruned_models/layers{depth}/trial{trial_n}/'
quantized_model_path    = f'../saved_model/quantized_models/layers{depth}/trial{trial_n}/'
parameter_csv_path      = f'../model-stats/layers-{depth}_filters-1-16_n-trail{trial_n}.csv'
plot_file_name          = f'../images/layers-{depth}_trial{trial_n}_baseline-pruned-and-quantized-model-size-vs-accuracy.png'

### Imports

In [ ]:
import os
import tempfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import matplotlib.pyplot as plt

### Load in the MFCC File

In [ ]:
from compression_lib import load_cnn_json

X, y, L = load_cnn_json(mfccs_json_path)
print(f"mapping the marine mammals: {L}")

### Prepare the Datasets

In [ ]:
# create train, validation and test sets
from compression_lib import prepare_datasets

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(X, y, 0.6, 0.5) # test size, vailidation size
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

### Load in Saved Models

In [ ]:
model_list = []
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(saved_model_path)):
    for f in filenames:
        loaded_model = tf.keras.models.load_model(dirpath+f)
        model_list.append(loaded_model)
print(f'loaded in {len(model_list)} baseline models')

### Determine Baseline Model Accuracies

In [ ]:
model_accuracy = []
for model in model_list:
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    model_accuracy.append(test_accuracy)
print(f'determined {len(model_accuracy)} accuracy values with max. accuracy: {max(model_accuracy)} and min accuracy: {min(model_accuracy)}')

### Establish Pruned Model Settings

In [ ]:
# create a prune_low_magnitude object
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

### Generate List of Pruned Models

In [ ]:
pruned_model_list = []
for model in model_list:
    model_for_pruning = prune_low_magnitude(model, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    pruned_model_list.append(model_for_pruning)
print(f'created {len(pruned_model_list)} pruned models')

### Train the Pruned Models

In [ ]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
]

model_history = []
for model in pruned_model_list:
  history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
                      validation_split=validation_split, callbacks=callbacks, verbose=verbosity)
  model_history.append(history)

### Get Accuracy values for Pruned Models

In [ ]:
pruned_model_accuracy = []
for model in pruned_model_list:
   _, model_for_pruning_accuracy = model.evaluate(X_test, y_test, verbose=0)
   pruned_model_accuracy.append(model_for_pruning_accuracy)

### Save the Pruned Models

In [ ]:
for dense_model_name, pruned_model in zip(filenames, pruned_model_list):
    model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)
    pruned_keras_file = pruned_model_path + dense_model_name
    tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
    print('Saved pruned Keras model to:', pruned_keras_file)

### Get list of Baseline Model Memory Size

In [ ]:
from compression_lib import get_gzipped_model_size

baseline_model_sizes = []
for f in filenames:
    baseline_model_path = saved_model_path + f
    model_size = get_gzipped_model_size(baseline_model_path)
    baseline_model_sizes.append(model_size)
print(baseline_model_sizes)

### Get List of Pruned Model Memory Size

In [ ]:
from compression_lib import get_gzipped_model_size

pruned_model_sizes = []
for f in filenames:
    model_path = pruned_model_path + f
    model_size = get_gzipped_model_size(model_path)
    pruned_model_sizes.append(model_size)
print(pruned_model_sizes)

### Plot Baseline and Pruned Curves

In [ ]:

plt.plot(baseline_model_sizes, model_accuracy, 'o', color='blue')
plt.plot(pruned_model_sizes, pruned_model_accuracy, 'o', color='red')
plt.xlabel("Number of Bytes")
plt.ylabel("Model Accuracy")

In [ ]:
import pandas as pd  
aa = pd.read_csv(parameter_csv_path)  
aa["Baseline Size"] = baseline_model_sizes[:16]
aa["Pruned Accuracy"] = pruned_model_accuracy[:16]
aa["Pruned Size"] = pruned_model_sizes[:16]
aa.to_csv(parameter_csv_path)

### Introduce Quantization

In [ ]:
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(X_test):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == y_test).mean()
  return accuracy

In [ ]:
"""
# must make the folders:
|- quantized_models
|  |- layers3
|  |   |- trial1
|  |   |- trial2
|  |   |- trial3
|  |- layers5
|  |   |- trial1
|  |   |- trial2
|  |   |- trial3
|  |- layers7
|  |   |- trial1
|  |   |- trial2
|  |   |- trial3
"""

quantized_model_list = []
for dense_model_name, pruned_model in zip(filenames, pruned_model_list):
  model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)
  quantized_keras_path = quantized_model_path + dense_model_name[:len(dense_model_name)-3] + ".tflite"
  converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  quantized_and_pruned_tflite_model = converter.convert()

  with open(quantized_keras_path, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)
  print('Saved quantized and pruned TFLite model to:', quantized_keras_path)
  quantized_model_list.append(quantized_and_pruned_tflite_model)


In [ ]:
import numpy as np
quantized_model_accuracy = []
for quantized_model in quantized_model_list:
    interpreter = tf.lite.Interpreter(model_content=quantized_model)
    interpreter.allocate_tensors()
    test_accuracy = evaluate_model(interpreter)
    quantized_model_accuracy.append(test_accuracy)

In [ ]:
print(quantized_model_accuracy)

In [ ]:
from compression_lib import get_gzipped_model_size

quantized_model_sizes = []
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(quantized_model_path)):
    for f in filenames:
        model_path = quantized_model_path + f
        model_size = get_gzipped_model_size(model_path)
        quantized_model_sizes.append(model_size)
print(quantized_model_sizes)

In [ ]:
plt.plot(baseline_model_sizes, model_accuracy, 'o', color='blue')
plt.plot(pruned_model_sizes, pruned_model_accuracy, 'o', color='red')
plt.plot(quantized_model_sizes, quantized_model_accuracy, 'o', color='green')
plt.xlabel("Number of Bytes")
plt.ylabel("Model Accuracy")
plt.legend(["Baseline Models", "Pruned Models", "Pruned and Quantized Models"])
plt.savefig(plot_file_name)

In [ ]:
import pandas as pd  
aa = pd.read_csv(parameter_csv_path)  
aa["Quantized Accuracy"] = quantized_model_accuracy[:16]
aa["Quantized Size"] = quantized_model_sizes[:16]
aa.to_csv(parameter_csv_path)

In [8]:
import numpy as np
import tensorflow as tf
import pathlib
from compression_lib import load_cnn_json

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="../saved_model/gui_models/arduino_mega.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()


# need to reshape the data 
signal, y, l = load_cnn_json("../mfccs_gui_test.json")
signal = np.float32(signal)[0]
signal = signal[:,:, np.newaxis]

interpreter.set_tensor(input_details[0]['index'], [signal])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print("the output is {}".format(output_data))

[{'name': 'serving_default_conv2d_30_input:0', 'index': 0, 'shape': array([  1, 130,  13,   1], dtype=int32), 'shape_signature': array([ -1, 130,  13,   1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
(130, 13, 1)
the output is [[1.6453000e-06 1.8397979e-05 1.2731196e-04 1.1345492e-03 9.9871814e-01
  2.7908094e-08 9.6622665e-10]]
